In [1]:
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output
import plotly.express as px

In [2]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')
df.sample(15)

,Country Name,Indicator Name,Year,Value
3102,Solomon Islands,Imports of goods and services (% of GDP),1962,NaN
5320,Gabon,"Agriculture, value added (% of GDP)",1967,NaN
12645,Faroe Islands,Electric power consumption (kWh per capita),1977,NaN
27864,Italy,Energy use (kg of oil equivalent per capita),1997,2834.402946
36185,Rwanda,"Industry, value added (% of GDP)",2007,12.397094
11806,Andorra,Energy use (kg of oil equivalent per capita),1977,NaN
13202,Kosovo,"Agriculture, value added (% of GDP)",1977,NaN
16916,Kuwait,Energy use (kg of oil equivalent per capita),1982,7548.129347
14545,Turkmenistan,"Services, etc., value added (% of GDP)",1977,NaN
12639,Ethiopia,"Life expectancy at birth, total (years)",1977,44.092146


In [3]:
df.shape

(36960, 4)

In [4]:
df['Indicator Name'].unique()

array(['Agriculture, value added (% of GDP)',
       'CO2 emissions (metric tons per capita)',
       'Domestic credit provided by financial sector (% of GDP)',
       'Electric power consumption (kWh per capita)',
       'Energy use (kg of oil equivalent per capita)',
       'Exports of goods and services (% of GDP)',
       'Fertility rate, total (births per woman)',
       'GDP growth (annual %)',
       'Imports of goods and services (% of GDP)',
       'Industry, value added (% of GDP)',
       'Inflation, GDP deflator (annual %)',
       'Life expectancy at birth, total (years)',
       'Population density (people per sq. km of land area)',
       'Services, etc., value added (% of GDP)'], dtype=object)

In [10]:
app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    
    html.Div([
        html.H1('Simulador prueba', style={'text-align': 'center'}),
        
        html.Img(src='https://upload.wikimedia.org/wikipedia/commons/thumb/b/b8/Banco_Santander_Logotipo.svg/238px-Banco_Santander_Logotipo.svg.png',
                 style={'position': 'absolute', 'top': '0', 'right': '0','padding-top': '40px','padding-right': '20px'}),
    
        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='crossfilter-xaxis-column'),
            
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-xaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '25px'})],
            style={'width': '49%', 'display': 'inline-block'}), # Para que llegue hasta la mitad el div
        
        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='crossfilter-yaxis-column'),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '25px'})],
            style={'width': '49%', 'float': 'right', 'display': 'inline-block'})],
        style={'padding': '30px 10px'}), #Deja un espacio Arriba

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter', hoverData={'points': [{'customdata': 'Japan'}]})],
            style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),],
        style={'display': 'inline-block', 'width': '49%'}),
    
    html.Div(dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='crossfilter-year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
    
]) 
            
#aca termina el dise;o Dashboard

@app.callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))     
    
def update_graph(xaxis_column_name, yaxis_column_name, xaxis_type, yaxis_type, year_value):
    
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])
    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')
    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')
    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')
    
    return fig
    
def create_time_series(dff, axis_type, title):

    fig = px.scatter(dff, x='Year', y='Value')
    fig.update_traces(mode='lines+markers')
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)
    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig    

@app.callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))

def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))

def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)  

In [ ]:
app.run_server(port=8052)

Dash is running on http://127.0.0.1:8052/

Dash is running on http://127.0.0.1:8052/

Dash is running on http://127.0.0.1:8052/

Dash is running on http://127.0.0.1:8052/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [09/May/2023 19:57:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 19:57:50] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [09/May/2023 19:57:50] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [09/May/2023 19:57:50] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [09/May/2023 19:57:50] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [09/May/2023 19:57:50] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 19:57:50] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 19:57:50] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [09/May/2023 19:57:50] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [09/May/2023 19:57:50] "GET /_dash-component-suites/dash/dcc/async-sli

127.0.0.1 - - [09/May/2023 20:00:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 20:00:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 20:00:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 20:00:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 20:00:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 20:00:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 20:00:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 20:00:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 20:00:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 20:00:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 20:00:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2023 20:00:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.